# PARTIE 3 - Entraînement du modèle de Machine Learning

## Mission : Créer un modèle capable de prédire le vainqueur d'un combat

Nous allons utiliser des algorithmes de **régression** pour prédire le pourcentage de victoire d'un Pokémon.

### Algorithmes testés :
1. **Régression Linéaire** - Modélise les relations entre variables
2. **Arbres de Décision** - Crée un arbre de décisions basé sur les caractéristiques
3. **Forêts Aléatoires** - Combine plusieurs arbres de décision pour améliorer la précision

### Objectif :
Trouver l'algorithme qui donne la meilleure précision pour prédire les victoires !

## 1. Import des modules nécessaires

Import des modules pour l'apprentissage automatique.

In [1]:
import pandas as pd
import numpy as np
import joblib

# Modules Scikit-Learn pour le Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

print("Modules importés avec succès !")

Modules importés avec succès !


## 2. Chargement du dataset

Chargement du fichier `dataset.csv` créé lors de la Partie 2.

In [2]:
# Chargement du dataset avec pour séparateur des tabulations
dataset = pd.read_csv("datas/dataset.csv", delimiter='\t')

# Suppression des lignes ayant des valeurs manquantes
dataset = dataset.dropna(axis=0, how='any')

print(f"Dataset chargé : {len(dataset)} Pokémons")
print(f"Colonnes disponibles : {len(dataset.columns)}")
print("\nAperçu des données :")
print(dataset.head())

Dataset chargé : 404 Pokémons
Colonnes disponibles : 17

Aperçu des données :
   NUMERO              NOM TYPE_1  TYPE_2  POINTS_DE_VIE  NIVEAU_ATTAQUE  \
0       1       Bulbizarre  Herbe  Poison             45              49   
1       2       Herbizarre  Herbe  Poison             60              62   
2       3       Florizarre  Herbe  Poison             80              82   
3       4  Mega Florizarre  Herbe  Poison             80             100   
6       7        Dracaufeu    Feu     Vol             78              84   

   NIVEAU_DEFENSE  NIVEAU_ATTAQUE_SPECIALE  NIVEAU_DEFENSE_SPECIALE  VITESSE  \
0              49                       65                       65       45   
1              63                       80                       80       60   
2              83                      100                      100       80   
3             123                      122                      120       80   
6              78                      109                       

## 3. Préparation des données d'apprentissage

### Sélection des features (variables prédictives)

Nous allons sélectionner les caractéristiques qui permettront de prédire le pourcentage de victoire :
- **POINTS_DE_VIE** (colonne 4)
- **NIVEAU_ATTAQUE** (colonne 5)
- **NIVEAU_DEFENSE** (colonne 6)
- **NIVEAU_ATTAQUE_SPECIALE** (colonne 7)
- **NIVEAU_DEFENSE_SPECIALE** (colonne 8)
- **VITESSE** (colonne 9)
- **GENERATION** (colonne 10)
- **LEGENDAIRE** (colonne 11)

La variable cible (à prédire) est **POURCENTAGE_DE_VICTOIRES**.

In [3]:
# Affichage des colonnes pour vérifier les indices
print("Colonnes du dataset :")
for i, col in enumerate(dataset.columns):
    print(f"  {i}: {col}")

# X = Variables prédictives (features 4 à 11)
# POINTS_DE_VIE, NIVEAU_ATTAQUE, NIVEAU_DEFENSE, NIVEAU_ATTAQUE_SPECIALE,
# NIVEAU_DEFENSE_SPECIALE, VITESSE, GENERATION, LEGENDAIRE
X = dataset.iloc[:, 4:12].values

# y = Variable cible (POURCENTAGE_DE_VICTOIRES)
# Trouver l'index de la colonne POURCENTAGE_DE_VICTOIRES
col_index = dataset.columns.get_loc('POURCENTAGE_DE_VICTOIRES')
y = dataset.iloc[:, col_index].values

print(f"\nVariables prédictives (X) : {X.shape}")
print(f"Variable cible (y) : {y.shape}")
print(f"\nExemple de données X (premier Pokémon) :")
print(X[0])
print(f"\nExemple de donnée y (pourcentage de victoire) : {y[0]}")

Colonnes du dataset :
  0: NUMERO
  1: NOM
  2: TYPE_1
  3: TYPE_2
  4: POINTS_DE_VIE
  5: NIVEAU_ATTAQUE
  6: NIVEAU_DEFENSE
  7: NIVEAU_ATTAQUE_SPECIALE
  8: NIVEAU_DEFENSE_SPECIALE
  9: VITESSE
  10: NOMBRE_GENERATIONS
  11: LEGENDAIRE
  12: Premier_Pokemon
  13: Second_Pokemon
  14: NBR_COMBATS
  15: NBR_VICTOIRES
  16: POURCENTAGE_DE_VICTOIRES

Variables prédictives (X) : (404, 8)
Variable cible (y) : (404,)

Exemple de données X (premier Pokémon) :
[45 49 49 65 65 45  1  0]

Exemple de donnée y (pourcentage de victoire) : 0.2781954887218045


## 4. Découpage en jeu d'apprentissage et jeu de tests

Nous allons diviser nos données en :
- **80% pour l'apprentissage** (X_APPRENTISSAGE, Y_APPRENTISSAGE)
- **20% pour les tests** (X_VALIDATION, Y_VALIDATION)

Cela permet de valider que le modèle fonctionne sur des données qu'il n'a jamais vues.

In [4]:
# Construction du jeu d'entraînement (80%) et du jeu de tests (20%)
X_APPRENTISSAGE, X_VALIDATION, Y_APPRENTISSAGE, Y_VALIDATION = train_test_split(
    X, y, test_size=0.2, random_state=0
)

print("Découpage effectué :")
print(f"  - Données d'apprentissage : {len(X_APPRENTISSAGE)} Pokémons ({len(X_APPRENTISSAGE)/len(X)*100:.1f}%)")
print(f"  - Données de validation : {len(X_VALIDATION)} Pokémons ({len(X_VALIDATION)/len(X)*100:.1f}%)")
print(f"\nDimensions :")
print(f"  - X_APPRENTISSAGE : {X_APPRENTISSAGE.shape}")
print(f"  - Y_APPRENTISSAGE : {Y_APPRENTISSAGE.shape}")
print(f"  - X_VALIDATION : {X_VALIDATION.shape}")
print(f"  - Y_VALIDATION : {Y_VALIDATION.shape}")

Découpage effectué :
  - Données d'apprentissage : 323 Pokémons (80.0%)
  - Données de validation : 81 Pokémons (20.0%)

Dimensions :
  - X_APPRENTISSAGE : (323, 8)
  - Y_APPRENTISSAGE : (323,)
  - X_VALIDATION : (81, 8)
  - Y_VALIDATION : (81,)


## 5. Test des algorithmes de Machine Learning

### Algorithme 1 : Régression Linéaire

La régression linéaire modélise les relations entre plusieurs variables prédictives et une variable cible.

In [5]:
# ---- ALGORITHME 1: REGRESSION LINEAIRE -----

# Choix de l'algorithme
algorithme_rl = LinearRegression()

# Apprentissage à l'aide de la fonction fit
# Apprentissage effectué sur le jeu de données d'apprentissage
algorithme_rl.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Réalisation de prédictions sur le jeu de tests (validation)
predictions_rl = algorithme_rl.predict(X_VALIDATION)

# Calcul de la précision sur les données d'apprentissage (pour détecter le surapprentissage)
precision_apprentissage_rl = algorithme_rl.score(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Calcul de la précision de l'apprentissage à l'aide de la
# fonction r2_score en comparant les valeurs prédites
# (predictions) et les valeurs attendues (Y_VALIDATION)
precision_rl = r2_score(Y_VALIDATION, predictions_rl)

print("=" * 60)
print(">> ----------- REGRESSION LINEAIRE -----------")
print(f">> Précision sur les données d'APPRENTISSAGE : {precision_apprentissage_rl:.4f} ({precision_apprentissage_rl*100:.2f}%)")
print(f">> Précision sur les données de VALIDATION : {precision_rl:.4f} ({precision_rl*100:.2f}%)")
print("=" * 60)

>> ----------- REGRESSION LINEAIRE -----------
>> Précision sur les données d'APPRENTISSAGE : 0.9201 (92.01%)
>> Précision sur les données de VALIDATION : 0.9044 (90.44%)


### Algorithme 2 : Arbres de Décision

Les arbres de décision créent un modèle de décision basé sur les caractéristiques des Pokémons.

In [6]:
# ---- ALGORITHME 2: ARBRES DE DECISIONS -----

# Choix de l'algorithme
algorithme_ad = DecisionTreeRegressor()

# Apprentissage
algorithme_ad.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Prédictions
predictions_ad = algorithme_ad.predict(X_VALIDATION)

# Calcul des précisions
precision_apprentissage_ad = algorithme_ad.score(X_APPRENTISSAGE, Y_APPRENTISSAGE)
precision_ad = r2_score(Y_VALIDATION, predictions_ad)

print("=" * 60)
print(">> ----------- ARBRES DE DECISIONS -----------")
print(f">> Précision sur les données d'APPRENTISSAGE : {precision_apprentissage_ad:.4f} ({precision_apprentissage_ad*100:.2f}%)")
print(f">> Précision sur les données de VALIDATION : {precision_ad:.4f} ({precision_ad*100:.2f}%)")
print("=" * 60)

>> ----------- ARBRES DE DECISIONS -----------
>> Précision sur les données d'APPRENTISSAGE : 0.9998 (99.98%)
>> Précision sur les données de VALIDATION : 0.8826 (88.26%)


### Algorithme 3 : Forêts Aléatoires (Random Forest)

Les forêts aléatoires combinent plusieurs arbres de décision pour améliorer la précision et éviter le surapprentissage.

In [7]:
# ---- ALGORITHME 3: FORETS ALEATOIRES (Random Forest) -----

# Choix de l'algorithme
algorithme_rf = RandomForestRegressor(random_state=0)

# Apprentissage
algorithme_rf.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Prédictions
predictions_rf = algorithme_rf.predict(X_VALIDATION)

# Calcul des précisions
precision_apprentissage_rf = algorithme_rf.score(X_APPRENTISSAGE, Y_APPRENTISSAGE)
precision_rf = r2_score(Y_VALIDATION, predictions_rf)

print("=" * 60)
print(">> ----------- FORETS ALEATOIRES -----------")
print(f">> Précision sur les données d'APPRENTISSAGE : {precision_apprentissage_rf:.4f} ({precision_apprentissage_rf*100:.2f}%)")
print(f">> Précision sur les données de VALIDATION : {precision_rf:.4f} ({precision_rf*100:.2f}%)")
print("=" * 60)

>> ----------- FORETS ALEATOIRES -----------
>> Précision sur les données d'APPRENTISSAGE : 0.9914 (99.14%)
>> Précision sur les données de VALIDATION : 0.9347 (93.47%)


## 6. Comparaison des résultats

Comparons les performances des trois algorithmes.

In [8]:
# Tableau comparatif des résultats
import pandas as pd

resultats = pd.DataFrame({
    'Algorithme': ['Régression Linéaire', 'Arbres de Décision', 'Forêts Aléatoires'],
    'Précision Apprentissage': [
        f"{precision_apprentissage_rl:.4f} ({precision_apprentissage_rl*100:.2f}%)",
        f"{precision_apprentissage_ad:.4f} ({precision_apprentissage_ad*100:.2f}%)",
        f"{precision_apprentissage_rf:.4f} ({precision_apprentissage_rf*100:.2f}%)"
    ],
    'Précision Validation': [
        f"{precision_rl:.4f} ({precision_rl*100:.2f}%)",
        f"{precision_ad:.4f} ({precision_ad*100:.2f}%)",
        f"{precision_rf:.4f} ({precision_rf*100:.2f}%)"
    ],
    'Score Validation': [precision_rl, precision_ad, precision_rf]
})

print("\n" + "=" * 80)
print("TABLEAU COMPARATIF DES ALGORITHMES")
print("=" * 80)
print(resultats[['Algorithme', 'Précision Apprentissage', 'Précision Validation']].to_string(index=False))
print("=" * 80)

# Déterminer le meilleur algorithme
meilleur_idx = resultats['Score Validation'].idxmax()
meilleur_algo = resultats.loc[meilleur_idx, 'Algorithme']
meilleur_score = resultats.loc[meilleur_idx, 'Score Validation']

print(f"\nMEILLEUR ALGORITHME : {meilleur_algo}")
print(f"   Précision : {meilleur_score:.4f} ({meilleur_score*100:.2f}%)")
print("=" * 80)


TABLEAU COMPARATIF DES ALGORITHMES
         Algorithme Précision Apprentissage Précision Validation
Régression Linéaire         0.9201 (92.01%)      0.9044 (90.44%)
 Arbres de Décision         0.9998 (99.98%)      0.8826 (88.26%)
  Forêts Aléatoires         0.9914 (99.14%)      0.9347 (93.47%)

MEILLEUR ALGORITHME : Forêts Aléatoires
   Précision : 0.9347 (93.47%)


## 7. Détection du surapprentissage (Overfitting)

### Qu'est-ce que le surapprentissage ?

Le **surapprentissage** (overfitting) se produit quand un modèle est trop adapté aux données d'apprentissage et ne se généralise pas bien à de nouvelles données.

**Indicateurs de surapprentissage** :
- Précision d'apprentissage très élevée (proche de 100%)
- Précision de validation beaucoup plus faible
- Grande différence entre les deux précisions

**Indicateurs d'un bon modèle** :
- Précisions d'apprentissage et de validation proches
- Précision de validation élevée

In [9]:
# Analyse du surapprentissage
print("=" * 80)
print("ANALYSE DU SURAPPRENTISSAGE")
print("=" * 80)

algorithmes_analyse = [
    ('Régression Linéaire', precision_apprentissage_rl, precision_rl),
    ('Arbres de Décision', precision_apprentissage_ad, precision_ad),
    ('Forêts Aléatoires', precision_apprentissage_rf, precision_rf)
]

for nom, prec_app, prec_val in algorithmes_analyse:
    difference = prec_app - prec_val
    print(f"\n{nom}:")
    print(f"  Précision Apprentissage: {prec_app*100:.2f}%")
    print(f"  Précision Validation: {prec_val*100:.2f}%")
    print(f"  Différence: {difference*100:.2f}%")
    
    if difference > 0.15:  # Plus de 15% de différence
        print(f"  ATTENTION : Possible surapprentissage détecté !")
    elif difference < 0.05:  # Moins de 5% de différence
        print(f"  Bon modèle : Généralisation excellente")
    else:
        print(f"  Modèle acceptable : Généralisation correcte")
print("\n" + "=" * 80)

ANALYSE DU SURAPPRENTISSAGE

Régression Linéaire:
  Précision Apprentissage: 92.01%
  Précision Validation: 90.44%
  Différence: 1.57%
  Bon modèle : Généralisation excellente

Arbres de Décision:
  Précision Apprentissage: 99.98%
  Précision Validation: 88.26%
  Différence: 11.72%
  Modèle acceptable : Généralisation correcte

Forêts Aléatoires:
  Précision Apprentissage: 99.14%
  Précision Validation: 93.47%
  Différence: 5.67%
  Modèle acceptable : Généralisation correcte



## 8. Sauvegarde du meilleur modèle

Sauvegardons le modèle avec la meilleure précision pour l'utiliser dans l'application de prédiction.

In [10]:
# Créer le répertoire modele s'il n'existe pas
import os
os.makedirs('modele', exist_ok=True)

# Sélection du meilleur algorithme basé sur la précision de validation
if precision_rf >= precision_rl and precision_rf >= precision_ad:
    meilleur_modele = algorithme_rf
    nom_modele = "Forêts Aléatoires"
elif precision_rl >= precision_ad:
    meilleur_modele = algorithme_rl
    nom_modele = "Régression Linéaire"
else:
    meilleur_modele = algorithme_ad
    nom_modele = "Arbres de Décision"

# Sauvegarde du modèle
fichier = 'modele/modele_pokemon.mod'
joblib.dump(meilleur_modele, fichier)

print("=" * 80)
print("SAUVEGARDE DU MODÈLE")
print("=" * 80)
print(f"Modèle sauvegardé : {nom_modele}")
print(f"Fichier : {fichier}")
print(f"Précision : {max(precision_rl, precision_ad, precision_rf)*100:.2f}%")
print("=" * 80)
print("\nLe modèle est prêt à être utilisé pour faire des prédictions !")

SAUVEGARDE DU MODÈLE
Modèle sauvegardé : Forêts Aléatoires
Fichier : modele/modele_pokemon.mod
Précision : 93.47%

Le modèle est prêt à être utilisé pour faire des prédictions !
